In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
#add the file to the spark session
spark.sparkContext.addFile(url)


In [6]:
# 2. Create a temporary view of the DataFrame.
home_salesDF = spark.read.csv(
    SparkFiles.get("home_sales_revised.csv"), sep=',', header=True
)
home_salesDF.show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [19]:
#make a new column where the price is properly shown
from pyspark.sql.functions import round
from pyspark.sql.functions import avg, max, min

home_salesDF = home_salesDF.withColumn("updated price", round(home_salesDF["price"],2))
home_salesDF.select(["price", "updated price"]).show()


+------+-------------+
| price|updated price|
+------+-------------+
|936923|     936923.0|
|379628|     379628.0|
|417866|     417866.0|
|239895|     239895.0|
|424418|     424418.0|
|218712|     218712.0|
|419199|     419199.0|
|323956|     323956.0|
|181925|     181925.0|
|258710|     258710.0|
|167864|     167864.0|
|337527|     337527.0|
|229896|     229896.0|
|210247|     210247.0|
|398667|     398667.0|
|437958|     437958.0|
|437375|     437375.0|
|288650|     288650.0|
|308313|     308313.0|
|177541|     177541.0|
+------+-------------+
only showing top 20 rows



In [22]:
home_salesDF.filter("bedrooms ==4").agg(avg("updated price")).show()

+------------------+
|avg(updated price)|
+------------------+
| 299661.0056283447|
+------------------+



In [25]:
from pyspark.sql.functions import month, year

In [39]:
#make a new DF w/year
home_salesDF.select(year(home_salesDF["date"])).show()


+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
|      2019|
|      2020|
|      2019|
|      2020|
|      2021|
|      2019|
|      2021|
|      2020|
|      2021|
|      2019|
|      2021|
|      2021|
|      2019|
|      2019|
|      2019|
+----------+
only showing top 20 rows



In [34]:
#create a new column for the year alone
home_salesDF = home_salesDF.withColumn("year", year(home_salesDF["date"]))
home_salesDF.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+-------------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|updated price|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+-------------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     936923.0|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     379628.0|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     417866.0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     239895.0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|  

In [41]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

home_salesDF.filter("bedrooms ==4").groupby("year").agg(round(avg("updated price"),2)).orderBy("year").show()

+----+----------------------------+
|year|round(avg(updated price), 2)|
+----+----------------------------+
|2019|                    300263.7|
|2020|                   298353.78|
|2021|                   301819.44|
|2022|                   296363.88|
+----+----------------------------+



In [44]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
home_salesDF.filter((home_salesDF["bedrooms"]==3)& (home_salesDF["bathrooms"]==3)).groupby("date_built").agg(round(avg("updated price"),2)).orderBy("date_built").show()


+----------+----------------------------+
|date_built|round(avg(updated price), 2)|
+----------+----------------------------+
|      2010|                   292859.62|
|      2011|                   291117.47|
|      2012|                   293683.19|
|      2013|                   295962.27|
|      2014|                   290852.27|
|      2015|                    288770.3|
|      2016|                   290555.07|
|      2017|                   292676.79|
+----------+----------------------------+



In [45]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
home_salesDF.filter((home_salesDF["bedrooms"]==3) & (home_salesDF["bathrooms"]==3) & (home_salesDF["floors"]==2) & (home_salesDF["sqft_living"]>=2000)).groupby("date_built").agg(round(avg("updated price"),2)).orderBy("date_built").show()



+----------+----------------------------+
|date_built|round(avg(updated price), 2)|
+----------+----------------------------+
|      2010|                   285010.22|
|      2011|                   276553.81|
|      2012|                   307539.97|
|      2013|                   303676.79|
|      2014|                   298264.72|
|      2015|                   297609.97|
|      2016|                    293965.1|
|      2017|                   280317.58|
+----------+----------------------------+



In [51]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
home_salesDF.filter(home_salesDF["updated price"]>350000).groupBy("view").agg(round(avg("updated price"),2).alias("average price")).show()


+----+-------------+
|view|average price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows



In [52]:
import time
#start the timer
start_time = time.time()

home_salesDF.filter(home_salesDF["updated price"]>350000).groupBy("view").agg(round(avg("updated price"),2).alias("average price")).show()

#stop the timer
end_time = time.time()
#runtime = end_time - start_time

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 0.6478960514068604 seconds ---


In [60]:
#save as a parquet file
home_salesDF.write.parquet('salesData', mode='overwrite')

In [61]:
home_salesDF.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+-------------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|updated price|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+-------------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     936923.0|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     379628.0|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     417866.0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     239895.0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|  

In [63]:
home_salesDF.createOrReplaceTempView("home_sales")

In [64]:
# 7. Cache the the temporary table home_sales.
spark.sql('CACHE TABLE home_sales')

DataFrame[]

In [65]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [66]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

home_salesDF.filter(home_salesDF["updated price"]>350000).groupBy("view").agg(round(avg("updated price"),2).alias("average price")).show()

#stop the timer
end_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 0.856564998626709 seconds ---


In [67]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
#make the view
home_salesDF.createGlobalTempView("buildData")

In [68]:
#save as a parquet file
home_salesDF.write.parquet('buildData', mode='overwrite')

In [69]:
# 11. Read the parquet formatted data.
buildParquetDF = spark.read.parquet('buildData')

In [70]:
# 12. Create a temporary table for the parquet data.
buildParquetDF.createOrReplaceTempView('buildParquetData')

In [72]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

buildParquetDF.filter(buildParquetDF["updated price"]>350000).groupBy("view").agg(round(avg("updated price"),2).alias("average price")).show()

#stop the timer
end_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 1.2381539344787598 seconds ---


In [73]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [74]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False